In [1]:
import pandas as pd
import fasttext
import fasttext.util

In [3]:
# 複勝の1行化

#!pip install optuna
fukusho = pd.read_csv('./fukusho20220909.csv')
groups = fukusho.groupby('レースID(新/馬番無)').groups

column_list = ['複勝配当', "馬番"]
new_df = pd.DataFrame()
max_length = 0
for group, indexes in groups.items():
    # 最後に並び替えをさせるのに最大作成された項目数を記録
    length = len(indexes)+1
    if length > max_length:
        max_length = length

    columns = list()
    values = list()
    columns += ['レースID(新/馬番無)']
    values += [fukusho.iloc[indexes]['レースID(新/馬番無)'].T.tolist()[0]]
#     values = values.sort_values(by='pred_time', ascending = True)
    # race_id = [target_column['race_id']]
    # columns += ['レースID(新/馬番無)']
    # values += [race_detail.iloc[indexes]['レースID(新/馬番無)'].T.tolist()[0]]

    for target_column in column_list:
        columns += [f'{target_column}_{x}' for x in range(1, length)]
        sort_values = fukusho.iloc[indexes, :]
        values += sort_values[target_column].T.tolist()
    
#     values.sort(reverse=True)
    record_df = pd.DataFrame([values], columns=columns)
    new_df = pd.concat([new_df, record_df], axis=0)
new_df.to_csv('./fukusho20220909_fukusho.csv')

In [36]:
# 血統情報取得
import pandas as pd
import time
from tqdm import tqdm

def scrape_peds(horse_id_list, pre_peds={}):
    peds = pre_peds
    for horse_id in tqdm(horse_id_list):
        if horse_id in peds.keys():
            continue
        try:
            url = "https://db.netkeiba.com/horse/ped/" + horse_id
            df = pd.read_html(url)[0]
            
            generations = {}
            for i in reversed(range(5)):
                generations[i] = df[i]
                df.drop([i], axis=1, inplace=True)
                df = df.drop_duplicates()
            
            ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)
            peds[horse_id] = ped.reset_index(drop=True)
            time.sleep(1)
        except IndexError:
            continue
        except:
            break
    return peds

def horse_data(horse_id_list, pre_peds={}):
    peds = pre_peds

In [37]:
horses = pd.read_csv('./horseid.csv')
targets = horses['horse_id'].astype(str)
peds = scrape_peds(targets)
peds = pd.concat([peds[horse_id] for horse_id in peds], axis=1).T
peds = peds.add_prefix('peds_')
peds.to_pickle('peds4.pickle')
p = pd.read_pickle('peds4.pickle')
p.to_csv('peds_1.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [01:13<00:00,  1.51s/it]


In [38]:
peds = pd.read_csv('./csv_new2/horses.csv')

ft_model = fasttext.load_model('ketto_model.bin')
def vec(x):
    try:
        sum = ft_model.get_sentence_vector(x).sum()
        if sum == 0:
            return None
        return sum
    except:
        return None

for i in range(1, 63):
    peds['peds_' + str(i)] = peds['peds' + str(i)].map(lambda x: vec(x))
    peds.drop(['peds' + str(i)], axis=1, inplace=True)

In [39]:
names = pd.read_csv('./csv_new2/horse_names.csv')

vec = names.merge(peds, on=['name']).fillna(0)

In [40]:
from sklearn.preprocessing import LabelEncoder

target_columns = [
    "peds_1", "peds_2","peds_3","peds_4","peds_5","peds_6","peds_7","peds_8","peds_9","peds_10",
    "peds_11","peds_12","peds_13","peds_14",
    "peds_15","peds_16","peds_17","peds_18","peds_19","peds_20",
    "peds_21","peds_22",
    "peds_23","peds_24","peds_25","peds_26","peds_27","peds_28","peds_29","peds_30",
    "peds_31","peds_32","peds_33","peds_34","peds_35","peds_36","peds_37","peds_38","peds_39","peds_40",
    "peds_41","peds_42","peds_43","peds_44","peds_45","peds_46","peds_47","peds_48","peds_49","peds_50",
    "peds_51","peds_52","peds_53","peds_54","peds_55","peds_56","peds_57","peds_58","peds_59",
    "peds_60","peds_61","peds_62"
]

df = vec.drop(["name"], axis=1)
for column in target_columns:
    df[column] = LabelEncoder().fit_transform(df[column].fillna('Na'))

for column in target_columns:
    df[column] = df[column].astype('category')

df.to_pickle('./pickle_new/peds_vec.pickle')

In [41]:
base = pd.read_csv('./csv_new2/base/race_detail_2.csv')

horse_sire_rate = pd.read_csv('./csv_new2/base/horse_sire_rate.csv')
horse_sire_rate.drop(['race_id', 'name', 'course', 'distance', 'date', 'place_id', 'start_time'], axis=1, inplace=True)
base['horse_race_id'] = base['id']

circle_rate = pd.read_csv('./csv_new2/base/circle_rate.csv')
new_race_trainings = base.merge(circle_rate, on='horse_race_id')

jockey_rate = pd.read_csv('./csv_new2/base/jockey_rate.csv')
jockey_rate.drop(['id', 'jockey_id', 'place_id', 'course', 'distance', 'date'], axis=1, inplace=True)
trainer_rate = pd.read_csv('./csv_new2/base/trainer_rate.csv')
trainer_rate.drop(['id', 'trainer_id', 'place_id', 'course', 'distance', 'date'], axis=1, inplace=True)
stallion_rate = pd.read_csv('./csv_new2/base/stallion_rate.csv')
stallion_rate.drop(['place_id', 'course', 'distance', 'date'], axis=1, inplace=True)

new_race_trainings_horses = new_race_trainings.merge(jockey_rate, how='left', on='horse_race_id')
new_race_trainings_horses = new_race_trainings_horses.merge(trainer_rate, how='left', on='horse_race_id')
new_race_trainings_horses = new_race_trainings_horses.merge(stallion_rate, how='left', on=['race_id', 'stallion_id'])

weather = pd.read_csv('./csv_new2/weathers.csv')
base_conc = new_race_trainings_horses.merge(weather, on='race_id')
base_conc['date'] = base_conc['date_y']
base_conc = base_conc.merge(horse_sire_rate, how='left', on='id')

seasons = pd.read_csv('./csv_new2/season_rate.csv')
b = base_conc.merge(seasons, how='left', on='id')

b.drop(['name', 'date_y', 'date_x', 'start_time'], axis=1, inplace=True)

horse_results_detail = pd.read_csv('./csv_new2/base/horse_results_detail.csv')
horse_results_detail = horse_results_detail.drop(['horse_id', 'age', 'course', 'distance', 'career', 'date', 'race_id'], axis=1)
b = b.merge(horse_results_detail, how='left', on='id')

In [6]:
standard_scaler = lambda  x: (x - x.mean()) / x.std()

targets = [
    '騎手全体勝率','騎手全体連対率','騎手全体複勝率','騎手競馬場別騎乗回数','騎手競馬場別勝率','騎手競馬場別連対率','騎手競馬場別複勝率',
    '騎手コース別騎乗回数','騎手コース別勝率','騎手コース別連対率','騎手コース別複勝率','騎手距離別騎乗回数','騎手距離別勝率','騎手距離別連対率',
    '騎手距離別複勝率','騎手同コース同距離別騎乗回数','騎手同コース同距離別勝率','騎手同コース同距離別連対率','騎手同コース同距離別複勝率',
    '調教師全体勝率','調教師全体連対率','調教師全体複勝率','調教師競馬場別騎乗回数','調教師競馬場別勝率','調教師競馬場別連対率',
    '調教師競馬場別複勝率','調教師コース別騎乗回数','調教師コース別勝率','調教師コース別連対率','調教師コース別複勝率','調教師距離別騎乗回数',
    '調教師距離別勝率','調教師距離別連対率','調教師距離別複勝率','調教師同コース同距離別騎乗回数','調教師同コース同距離別勝率',
    '調教師年齢別年間勝率', '調教師年齢別年間連対率', '調教師年齢別年間複勝率','調教師年齢別勝率', '調教師年齢別連対率', '調教師年齢別複勝率',
    '調教師同コース同距離別連対率','調教師同コース同距離別複勝率','種牡馬全体勝率','種牡馬全体連対率','種牡馬全体複勝率',
    '種牡馬競馬場別出走頭数','種牡馬競馬場別勝率','種牡馬競馬場別連対率','種牡馬競馬場別複勝率','種牡馬コース別出走頭数','種牡馬コース別勝率',
    '種牡馬コース別連対率','種牡馬コース別複勝率','種牡馬距離別出走頭数','種牡馬距離別勝率','種牡馬距離別連対率','種牡馬距離別複勝率',
    '種牡馬同コース同距離別出走頭数','種牡馬同コース同距離別勝率','種牡馬同コース同距離別連対率','種牡馬同コース同距離別複勝率',
    '種牡馬同周り勝率', '種牡馬同周り連対率', '種牡馬同周り複勝率',  '種牡馬同枠勝率', '種牡馬同枠連対率', '種牡馬同枠複勝率',
    '父系統出走頭数','父系統全体勝率','父系統全体連対率','父系統全体複勝率','父系統競馬場別出走頭数','父系統競馬場別勝率','父系統競馬場別連対率','父系統競馬場別複勝率','父系統コース別出走頭数','父系統コース別勝率',
    '父系統コース別連対率','父系統コース別複勝率','父系統距離別出走頭数','父系統距離別勝率','父系統距離別連対率','父系統距離別複勝率',
    '父系統同コース同距離別出走頭数','父系統同コース同距離別勝率','父系統同コース同距離別連対率','父系統同コース同距離別複勝率',
    '季節勝率', '季節連対率', '季節複勝率', '逃げ率','先行率','中団率','追込率','マクリ率','上がり3F平均',
    '勝率','同競馬場勝率','同距離勝率','同競馬場同距離勝率','同騎手騎乗勝率','コースタイプ勝率','同距離同クラス勝率','同枠タイプ生涯勝率',
    '連対率','同競馬場連対率','同距離連対率','同競馬場同距離連対率','同騎手騎乗連対率','コースタイプ連対率','同距離同クラス連対率','同枠タイプ生涯連対率',
    '複勝率','同競馬場複勝率','同距離複勝率','同競馬場同距離複勝率','同騎手騎乗複勝率','コースタイプ複勝率','同距離同クラス複勝率','同枠タイプ生涯複勝率',
    '生涯出遅れ率','騎乗騎手年間出遅れ率', '同周り勝率', '同周り連対率', '同周り複勝率'
]

# b = b.set_index('race_id')
for t in targets:
    b[t + '偏差'] = b.groupby('race_id')[t].transform(standard_scaler)
    
# b.drop(targets, axis=1, inplace=True)

cp_all_rb = b.copy()
ind = ['先行指数', 'ペース指数', '上がり指数', 'スピード指数']
for i in range(1, 6):
    for idx in ind:
        cp_all_rb[str(i) + '走前'+ idx + '偏差'] = cp_all_rb.groupby(level=0)[str(i) + '走前'+ idx].transform(standard_scaler)
        
b_conc = cp_all_rb.reset_index()
b_copy = b_conc.copy()

b_copy.to_pickle('./pickle_new/base_race_20221016.pickle')

/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_74852/3654046672.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  b[t + '偏差'] = b.groupby('race_id')[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_74852/3654046672.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  b[t + '偏差'] = b.groupby('race_id')[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_74852/3654046672.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is us

/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_74852/3654046672.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  b[t + '偏差'] = b.groupby('race_id')[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_74852/3654046672.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  b[t + '偏差'] = b.groupby('race_id')[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_74852/3654046672.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [71]:
# 新馬戦
standard_scaler = lambda  x: (x - x.mean()) / x.std()  * 10 + 50

base = pd.read_csv('./csv_new2/base/race_detail_shinba.csv')
horse_sire_rate = pd.read_csv('./csv_new2/base/horse_sire_rate.csv')

horse_sire_rate.drop(['race_id', 'name', 'course', 'distance', 'date', 'place_id', 'start_time'], axis=1, inplace=True)

base['horse_race_id'] = base['id']

jockey_rate = pd.read_csv('./csv_new2/base/jockey_rate.csv')
jockey_rate.drop(['id', 'jockey_id', 'place_id', 'course', 'distance', 'date'], axis=1, inplace=True)
trainer_rate = pd.read_csv('./csv_new2/base/trainer_rate.csv')
trainer_rate.drop(['id', 'trainer_id', 'place_id', 'course', 'distance', 'date'], axis=1, inplace=True)
stallion_rate = pd.read_csv('./csv_new2/base/stallion_rate.csv')
stallion_rate.drop(['place_id', 'course', 'distance', 'date'], axis=1, inplace=True)

new_race_trainings_horses = base.merge(jockey_rate, how='left', on='horse_race_id')
new_race_trainings_horses = new_race_trainings_horses.merge(trainer_rate, how='left', on='horse_race_id')
new_race_trainings_horses = new_race_trainings_horses.merge(stallion_rate, how='left', on=['race_id', 'stallion_id'])

new_race_trainings_horses.drop(['result_except'], axis=1, inplace=True)

weather = pd.read_csv('./csv_new2/weathers.csv')
base_conc = new_race_trainings_horses.merge(weather, on='race_id')
base_conc['date'] = base_conc['date_y']
base_conc.drop(['date_y', 'date_x', 'start_time'], axis=1, inplace=True)
base_conc = base_conc.merge(horse_sire_rate, how='left', on='id')

seasons = pd.read_csv('./csv_new2/season_rate.csv')
b = base_conc.merge(seasons, how='left', on='id')

b.drop(['name', 'sireline'], axis=1, inplace=True)

horse_results_detail = pd.read_csv('./csv_new2/base/horse_results_detail.csv')
horse_results_detail = horse_results_detail.drop(['horse_id', 'age', 'course', 'distance', 'career', 'date', 'race_id'], axis=1)
b = b.merge(horse_results_detail, how='left', on='id')

standard_scaler = lambda  x: (x - x.mean()) / x.std()

targets = [
    '騎手全体勝率','騎手全体連対率','騎手全体複勝率','騎手競馬場別騎乗回数','騎手競馬場別勝率','騎手競馬場別連対率','騎手競馬場別複勝率',
    '騎手コース別騎乗回数','騎手コース別勝率','騎手コース別連対率','騎手コース別複勝率','騎手距離別騎乗回数','騎手距離別勝率','騎手距離別連対率',
    '騎手距離別複勝率','騎手同コース同距離別騎乗回数','騎手同コース同距離別勝率','騎手同コース同距離別連対率','騎手同コース同距離別複勝率',
    '調教師全体勝率','調教師全体連対率','調教師全体複勝率','調教師競馬場別騎乗回数','調教師競馬場別勝率','調教師競馬場別連対率',
    '調教師競馬場別複勝率','調教師コース別騎乗回数','調教師コース別勝率','調教師コース別連対率','調教師コース別複勝率','調教師距離別騎乗回数',
    '調教師距離別勝率','調教師距離別連対率','調教師距離別複勝率','調教師同コース同距離別騎乗回数','調教師同コース同距離別勝率',
    '調教師年齢別年間勝率', '調教師年齢別年間連対率', '調教師年齢別年間複勝率','調教師年齢別勝率', '調教師年齢別連対率', '調教師年齢別複勝率',
    '調教師同コース同距離別連対率','調教師同コース同距離別複勝率','種牡馬全体勝率','種牡馬全体連対率','種牡馬全体複勝率',
    '種牡馬競馬場別出走頭数','種牡馬競馬場別勝率','種牡馬競馬場別連対率','種牡馬競馬場別複勝率','種牡馬コース別出走頭数','種牡馬コース別勝率',
    '種牡馬コース別連対率','種牡馬コース別複勝率','種牡馬距離別出走頭数','種牡馬距離別勝率','種牡馬距離別連対率','種牡馬距離別複勝率',
    '種牡馬同コース同距離別出走頭数','種牡馬同コース同距離別勝率','種牡馬同コース同距離別連対率','種牡馬同コース同距離別複勝率',
    '種牡馬同周り勝率', '種牡馬同周り連対率', '種牡馬同周り複勝率',  '種牡馬同枠勝率', '種牡馬同枠連対率', '種牡馬同枠複勝率',
    '父系統出走頭数','父系統全体勝率','父系統全体連対率','父系統全体複勝率','父系統競馬場別出走頭数','父系統競馬場別勝率','父系統競馬場別連対率','父系統競馬場別複勝率','父系統コース別出走頭数','父系統コース別勝率',
    '父系統コース別連対率','父系統コース別複勝率','父系統距離別出走頭数','父系統距離別勝率','父系統距離別連対率','父系統距離別複勝率',
    '父系統同コース同距離別出走頭数','父系統同コース同距離別勝率','父系統同コース同距離別連対率','父系統同コース同距離別複勝率',
    '季節勝率', '季節連対率', '季節複勝率', '逃げ率','先行率','中団率','追込率','マクリ率','上がり3F平均',
    '勝率','同競馬場勝率','同距離勝率','同競馬場同距離勝率','同騎手騎乗勝率','コースタイプ勝率','同距離同クラス勝率','同枠タイプ生涯勝率',
    '連対率','同競馬場連対率','同距離連対率','同競馬場同距離連対率','同騎手騎乗連対率','コースタイプ連対率','同距離同クラス連対率','同枠タイプ生涯連対率',
    '複勝率','同競馬場複勝率','同距離複勝率','同競馬場同距離複勝率','同騎手騎乗複勝率','コースタイプ複勝率','同距離同クラス複勝率','同枠タイプ生涯複勝率',
    '生涯出遅れ率','騎乗騎手年間出遅れ率'
]

b = b.set_index('race_id')
for t in targets:
    b[t + '偏差'] = b.groupby(level=0)[t].transform(standard_scaler)
    
b.drop(targets, axis=1, inplace=True)
b.drop(['horse_race_id'], axis=1, inplace=True)
b.to_pickle('./pickle_new/shinba_base.pickle')

/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_38971/4152359629.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  b[t + '偏差'] = b.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_38971/4152359629.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  b[t + '偏差'] = b.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_38971/4152359629.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_38971/4152359629.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  b[t + '偏差'] = b.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_38971/4152359629.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  b[t + '偏差'] = b.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_38971/4152359629.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_38971/4152359629.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  b[t + '偏差'] = b.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_38971/4152359629.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  b[t + '偏差'] = b.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_38971/4152359629.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [77]:
dd = b_copy.drop(['逃げ率偏差値','先行率偏差値','中団率偏差値','追込率偏差値','マクリ率偏差値','上がり3F平均偏差値','獲得賞金合計偏差値','付加賞金合計偏差値'], axis=1)
horse_results_detail = pd.read_csv('./csv_new2/base/horse_results_detail.csv')
horse_results_detail = horse_results_detail.drop(['horse_id', 'age', 'course', 'distance', 'career', 'date', 'race_id'], axis=1)
dd = dd.merge(horse_results_detail[['id', '逃げ率','先行率','中団率','追込率','マクリ率','上がり3F平均','獲得賞金合計','付加賞金合計']], how='left', on='id')
dd


,race_id,id,horse_id,age,course,distance,result,jockey_id,multiple,affiliation_id,...,生涯出遅れ率偏差値,騎乗騎手年間出遅れ率偏差値,逃げ率,先行率,中団率,追込率,マクリ率,上がり3F平均,獲得賞金合計,付加賞金合計
0,2007090806040101,200709080604010101,2005100316,2,2,1200,6,629,1,1,...,59.383576,53.461260,0.0000,0.9999,0.0000,0.0000,0.0,38.9,255.0,NaN
1,2007090806040101,200709080604010105,2005100929,2,2,1200,2,679,0,1,...,44.567559,47.118061,0.0000,0.3333,0.3333,0.3333,0.0,37.0,50.0,NaN
2,2007090806040101,200709080604010104,2005101474,2,2,1200,1,836,0,1,...,44.567559,54.469572,0.5000,0.5000,0.0000,0.0000,0.0,35.9,410.0,NaN
3,2007090806040101,200709080604010102,2005101743,2,2,1200,3,711,0,1,...,74.199592,45.614760,0.0000,0.0000,0.9999,0.0000,0.0,36.9,0.0,NaN
4,2007090806040101,200709080604010108,2005103238,2,2,1200,5,699,0,1,...,44.567559,45.119770,0.0000,0.0000,0.9999,0.0000,0.0,36.8,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672511,2022080704020412,202208070402041213,2019105293,3,1,1600,8,1014,0,1,...,41.630455,32.190246,0.0000,0.9999,0.0000,0.0000,0.0,33.9,600.0,NaN
672512,2022080704020412,202208070402041208,2019105413,3,1,1600,7,1164,0,2,...,47.633050,63.977211,0.1667,0.5000,0.1667,0.1667,0.0,36.6,860.0,NaN
672513,2022080704020412,202208070402041203,2019105515,3,1,1600,5,5386,0,2,...,77.638822,42.679489,0.0000,0.0000,0.5000,0.5000,0.0,34.6,720.0,0.0
672514,2022080704020412,202208070402041214,2019105859,3,1,1600,2,1093,0,2,...,65.637233,56.923576,0.0000,0.3333,0.5000,0.1667,0.0,35.1,1730.0,3.8


In [42]:
import pandas as pd
# 新規レース追加
new_races = pd.read_csv('./csv_new2/20221127/horse_results.csv')

jockey_rate = pd.read_csv('./csv_new2/20221127/jockey_rate.csv')
jockey_rate.drop(['id', 'jockey_id', 'place_id', 'course', 'distance', 'date'], axis=1, inplace=True)
trainer_rate = pd.read_csv('./csv_new2/20221127/trainer_rate.csv')
trainer_rate.drop(['id', 'trainer_id', 'place_id', 'course', 'distance', 'date'], axis=1, inplace=True)
stallion_rate = pd.read_csv('./csv_new2/20221127/stallion_rate.csv')
stallion_rate.drop(['place_id', 'course', 'distance', 'date'], axis=1, inplace=True)
df_conc = new_races.merge(jockey_rate, how='left', on='horse_race_id')
df_conc = df_conc.merge(trainer_rate, how='left', on='horse_race_id')
df_conc = df_conc.merge(stallion_rate, how='left', on=['race_id', 'stallion_id'])

weather = pd.read_csv('./csv_new2/20221127/weathers.csv')
df_conc = df_conc.merge(weather, how='left', on='race_id')
df_conc['date'] = df_conc['date_y']
df_conc.drop(['date_y', 'date_x', 'start_time'], axis=1, inplace=True)

horse_results_detail = pd.read_csv('./csv_new2/20221127/race_detail.csv')
horse_results_detail = horse_results_detail.drop(['horse_id', 'age', 'course', 'distance', 'career', 'date', 'race_id'], axis=1)
df_conc['id'] = df_conc['horse_race_id']
df_conc = df_conc.merge(horse_results_detail, how='left', on='id')

d = df_conc.copy()
horse_sire_rate = pd.read_csv('./csv_new2/20221127/horse_sire_rate.csv')
horse_sire_rate.drop(['race_id', 'name', 'course', 'distance', 'date', 'place_id', 'start_time'], axis=1, inplace=True)
d = d.merge(horse_sire_rate, how='left', on='id')

seasons = pd.read_csv('./csv_new2/20221127/season_rate.csv')
seasons.drop(['horse_id'], axis=1, inplace=True)
d = d.merge(seasons, how='left', on='id')

d['date'] = d['date_x']
d.drop(['name', 'date_x', 'date_y'], axis=1, inplace=True)

In [43]:
standard_scaler = lambda  x: (x - x.mean()) / x.std()

targets = [
    '騎手全体勝率','騎手全体連対率','騎手全体複勝率','騎手競馬場別騎乗回数','騎手競馬場別勝率','騎手競馬場別連対率','騎手競馬場別複勝率',
    '騎手コース別騎乗回数','騎手コース別勝率','騎手コース別連対率','騎手コース別複勝率','騎手距離別騎乗回数','騎手距離別勝率','騎手距離別連対率',
    '騎手距離別複勝率','騎手同コース同距離別騎乗回数','騎手同コース同距離別勝率','騎手同コース同距離別連対率','騎手同コース同距離別複勝率',
    '調教師全体勝率','調教師全体連対率','調教師全体複勝率','調教師競馬場別騎乗回数','調教師競馬場別勝率','調教師競馬場別連対率',
    '調教師競馬場別複勝率','調教師コース別騎乗回数','調教師コース別勝率','調教師コース別連対率','調教師コース別複勝率','調教師距離別騎乗回数',
    '調教師距離別勝率','調教師距離別連対率','調教師距離別複勝率','調教師同コース同距離別騎乗回数','調教師同コース同距離別勝率',
    '調教師年齢別年間勝率', '調教師年齢別年間連対率', '調教師年齢別年間複勝率','調教師年齢別勝率', '調教師年齢別連対率', '調教師年齢別複勝率',
    '調教師同コース同距離別連対率','調教師同コース同距離別複勝率','種牡馬全体勝率','種牡馬全体連対率','種牡馬全体複勝率',
    '種牡馬競馬場別出走頭数','種牡馬競馬場別勝率','種牡馬競馬場別連対率','種牡馬競馬場別複勝率','種牡馬コース別出走頭数','種牡馬コース別勝率',
    '種牡馬コース別連対率','種牡馬コース別複勝率','種牡馬距離別出走頭数','種牡馬距離別勝率','種牡馬距離別連対率','種牡馬距離別複勝率',
    '種牡馬同コース同距離別出走頭数','種牡馬同コース同距離別勝率','種牡馬同コース同距離別連対率','種牡馬同コース同距離別複勝率',
    '種牡馬同周り勝率', '種牡馬同周り連対率', '種牡馬同周り複勝率',  '種牡馬同枠勝率', '種牡馬同枠連対率', '種牡馬同枠複勝率',
    '父系統出走頭数','父系統全体勝率','父系統全体連対率','父系統全体複勝率','父系統競馬場別出走頭数','父系統競馬場別勝率','父系統競馬場別連対率','父系統競馬場別複勝率','父系統コース別出走頭数','父系統コース別勝率',
    '父系統コース別連対率','父系統コース別複勝率','父系統距離別出走頭数','父系統距離別勝率','父系統距離別連対率','父系統距離別複勝率',
    '父系統同コース同距離別出走頭数','父系統同コース同距離別勝率','父系統同コース同距離別連対率','父系統同コース同距離別複勝率',
    '季節勝率', '季節連対率', '季節複勝率', '逃げ率','先行率','中団率','追込率','マクリ率','上がり3F平均',
    '勝率','同競馬場勝率','同距離勝率','同競馬場同距離勝率','同騎手騎乗勝率','コースタイプ勝率','同距離同クラス勝率','同枠タイプ生涯勝率',
    '連対率','同競馬場連対率','同距離連対率','同競馬場同距離連対率','同騎手騎乗連対率','コースタイプ連対率','同距離同クラス連対率','同枠タイプ生涯連対率',
    '複勝率','同競馬場複勝率','同距離複勝率','同競馬場同距離複勝率','同騎手騎乗複勝率','コースタイプ複勝率','同距離同クラス複勝率','同枠タイプ生涯複勝率',
    '生涯出遅れ率','騎乗騎手年間出遅れ率', '同周り勝率', '同周り連対率', '同周り複勝率'
]

d = d.set_index('race_id')
for t in targets:
    d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)

dcp_all_rb = d.copy()
ind = ['先行指数', 'ペース指数', '上がり指数', 'スピード指数']
for i in range(1, 6):
    for idx in ind:
        dcp_all_rb[str(i) + '走前'+ idx + '偏差'] = dcp_all_rb.groupby(level=0)[str(i) + '走前'+ idx].transform(standard_scaler)
        
d_conc = dcp_all_rb.reset_index()
d_copy = d_conc.copy()

# d_copy.drop(['ペース指数', '先行指数', 'スピード指数', '上がり指数'], axis=1, inplace=True)
d_copy.to_pickle('./pickle_new/new_race_20221127.pickle')

/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/2259639867.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/2259639867.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/2259639867.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/2259639867.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/2259639867.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/2259639867.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/2259639867.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/2259639867.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/2259639867.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [44]:
import pandas as pd
standard_scaler = lambda  x: (x - x.mean()) / x.std()

# 新規レース追加(新馬戦用)
new_races = pd.read_csv('./csv_new2/20221127/race_detail_shinba.csv')

jockey_rate = pd.read_csv('./csv_new2/20221127/jockey_rate.csv')
jockey_rate.drop(['id', 'jockey_id', 'place_id', 'course', 'distance', 'date'], axis=1, inplace=True)
trainer_rate = pd.read_csv('./csv_new2/20221127/trainer_rate.csv')
trainer_rate.drop(['id', 'trainer_id', 'place_id', 'course', 'distance', 'date'], axis=1, inplace=True)
stallion_rate = pd.read_csv('./csv_new2/20221127/stallion_rate.csv')
stallion_rate.drop(['place_id', 'course', 'distance', 'date'], axis=1, inplace=True)

df_conc = new_races.merge(jockey_rate, how='left', on='horse_race_id')
df_conc = df_conc.merge(trainer_rate, how='left', on='horse_race_id')
df_conc = df_conc.merge(stallion_rate, how='left', on=['race_id', 'stallion_id'])

df_conc.drop(['result_except'], axis=1, inplace=True)

weather = pd.read_csv('./csv_new2/20221127/weathers.csv')
df_conc = df_conc.merge(weather, how='left', on='race_id')
df_conc['date'] = df_conc['date_y']
df_conc['id'] = df_conc['horse_race_id']
df_conc.drop(['horse_race_id','date_y', 'date_x', 'start_time'], axis=1, inplace=True)

horse_results_detail = pd.read_csv('./csv_new2/20221127/race_detail.csv')
horse_results_detail = horse_results_detail.drop(['horse_id', 'age', 'course', 'distance', 'career', 'date', 'race_id'], axis=1)
df_conc = df_conc.merge(horse_results_detail, how='left', on='id')

d = df_conc.copy()
horse_sire_rate = pd.read_csv('./csv_new2/20221127/horse_sire_rate.csv')
horse_sire_rate.drop(['race_id', 'name', 'course', 'distance', 'date', 'place_id', 'start_time'], axis=1, inplace=True)
d = d.merge(horse_sire_rate, how='left', on='id')

seasons = pd.read_csv('./csv_new2/20221127/season_rate.csv')
seasons.drop(['horse_id'], axis=1, inplace=True)
d = d.merge(seasons, how='left', on='id')

d['date'] = d['date_x']
d.drop(['name', 'sireline', 'date_x', 'date_y'], axis=1, inplace=True)

targets = [
    '騎手全体勝率','騎手全体連対率','騎手全体複勝率','騎手競馬場別騎乗回数','騎手競馬場別勝率','騎手競馬場別連対率','騎手競馬場別複勝率',
    '騎手コース別騎乗回数','騎手コース別勝率','騎手コース別連対率','騎手コース別複勝率','騎手距離別騎乗回数','騎手距離別勝率','騎手距離別連対率',
    '騎手距離別複勝率','騎手同コース同距離別騎乗回数','騎手同コース同距離別勝率','騎手同コース同距離別連対率','騎手同コース同距離別複勝率',
    '調教師全体勝率','調教師全体連対率','調教師全体複勝率','調教師競馬場別騎乗回数','調教師競馬場別勝率','調教師競馬場別連対率',
    '調教師競馬場別複勝率','調教師コース別騎乗回数','調教師コース別勝率','調教師コース別連対率','調教師コース別複勝率','調教師距離別騎乗回数',
    '調教師距離別勝率','調教師距離別連対率','調教師距離別複勝率','調教師同コース同距離別騎乗回数','調教師同コース同距離別勝率',
    '調教師年齢別年間勝率', '調教師年齢別年間連対率', '調教師年齢別年間複勝率','調教師年齢別勝率', '調教師年齢別連対率', '調教師年齢別複勝率',
    '調教師同コース同距離別連対率','調教師同コース同距離別複勝率','種牡馬全体勝率','種牡馬全体連対率','種牡馬全体複勝率',
    '種牡馬競馬場別出走頭数','種牡馬競馬場別勝率','種牡馬競馬場別連対率','種牡馬競馬場別複勝率','種牡馬コース別出走頭数','種牡馬コース別勝率',
    '種牡馬コース別連対率','種牡馬コース別複勝率','種牡馬距離別出走頭数','種牡馬距離別勝率','種牡馬距離別連対率','種牡馬距離別複勝率',
    '種牡馬同コース同距離別出走頭数','種牡馬同コース同距離別勝率','種牡馬同コース同距離別連対率','種牡馬同コース同距離別複勝率',
    '種牡馬同周り勝率', '種牡馬同周り連対率', '種牡馬同周り複勝率',  '種牡馬同枠勝率', '種牡馬同枠連対率', '種牡馬同枠複勝率',
    '父系統出走頭数','父系統全体勝率','父系統全体連対率','父系統全体複勝率','父系統競馬場別出走頭数','父系統競馬場別勝率','父系統競馬場別連対率','父系統競馬場別複勝率','父系統コース別出走頭数','父系統コース別勝率',
    '父系統コース別連対率','父系統コース別複勝率','父系統距離別出走頭数','父系統距離別勝率','父系統距離別連対率','父系統距離別複勝率',
    '父系統同コース同距離別出走頭数','父系統同コース同距離別勝率','父系統同コース同距離別連対率','父系統同コース同距離別複勝率',
    '季節勝率', '季節連対率', '季節複勝率', '逃げ率','先行率','中団率','追込率','マクリ率','上がり3F平均',
    '勝率','同競馬場勝率','同距離勝率','同競馬場同距離勝率','同騎手騎乗勝率','コースタイプ勝率','同距離同クラス勝率','同枠タイプ生涯勝率',
    '連対率','同競馬場連対率','同距離連対率','同競馬場同距離連対率','同騎手騎乗連対率','コースタイプ連対率','同距離同クラス連対率','同枠タイプ生涯連対率',
    '複勝率','同競馬場複勝率','同距離複勝率','同競馬場同距離複勝率','同騎手騎乗複勝率','コースタイプ複勝率','同距離同クラス複勝率','同枠タイプ生涯複勝率',
    '生涯出遅れ率','騎乗騎手年間出遅れ率'
]

d = d.set_index('race_id')
for t in targets:
    d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
    
d.drop(targets, axis=1, inplace=True)

d = d.reset_index()
d.to_pickle('./pickle_new/new_race_20221127_shinba.pickle')

/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/3581091827.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/3581091827.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/3581091827.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/3581091827.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/3581091827.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d[t + '偏差'] = d.groupby(level=0)[t].transform(standard_scaler)
/var/folders/dn/99p8d8fn6g75hkllgmntdmxc0000gn/T/ipykernel_31913/3581091827.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [16]:
print((set(d_copy.keys()) - set(b.keys())))